In [1]:
# http://docs.graphene-python.org/en/latest/execution/dataloader/

import json
import graphene

print(graphene.__version__)

def unorder(ordered_dict):
    return json.dumps(ordered_dict, indent=4)

def run(query, request):
    schema = graphene.Schema(query=query)
    result = schema.execute(request)
    if result.errors:
        raise Exception(result.errors)
    print( unorder(result.data) )

2.0.1


In [2]:
class User(graphene.ObjectType):
    name = graphene.String()
    best_friend = graphene.Field(lambda: User)
    friends = graphene.Field(graphene.List(lambda: User))
    
    best_friend_id = 100
    friend_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9] 

    def resolve_best_friend(self, info):
        # user_loader will be defined in code below
        return user_loader.load(self.best_friend_id)

    def resolve_friends(self, info):
        return user_loader.load_many(self.friend_ids)
    
class Query(graphene.ObjectType):
    
    me = graphene.Field(User)

    def resolve_me(self, info):
        return User(name='abcd')


In [3]:
# get_users is THE function that send SQL query. It accept a list of ids and return a list if User who inherit ObjectType.

def get_users(ids):
    print(f"get_users({ids}) SELECT * FROM table_user WHERE id IN {ids}")
    return [User(name=f"User{id}") for id in ids]

In [4]:
from promise import Promise
from promise.dataloader import DataLoader

class UserLoader(DataLoader):
    def batch_load_fn(self, keys):
        # Here we return a promise that will result on the
        # corresponding user for each key in keys
        print(f'self.batch_load_fn({keys})')
        return Promise.resolve(get_users(ids=keys))

In [5]:
user_loader = UserLoader()

# print('====')
# user_loader.load(1).then(lambda user: user_loader.load(user.best_friend_id))
# print('====')
# user_loader.load(2).then(lambda user: user_loader.load(user.best_friend_id))


In [6]:
run(Query, """
{
  me {
    name
    bestFriend {
      name
    }
    friends {
      name
      bestFriend {
        name
      }
    }
  }
}
""")

self.batch_load_fn([100, 1, 2, 3, 4, 5, 6, 7, 8, 9])
get_users([100, 1, 2, 3, 4, 5, 6, 7, 8, 9]) SELECT * FROM table_user WHERE id IN [100, 1, 2, 3, 4, 5, 6, 7, 8, 9]
{
    "me": {
        "name": "abcd",
        "bestFriend": {
            "name": "User100"
        },
        "friends": [
            {
                "name": "User1",
                "bestFriend": {
                    "name": "User100"
                }
            },
            {
                "name": "User2",
                "bestFriend": {
                    "name": "User100"
                }
            },
            {
                "name": "User3",
                "bestFriend": {
                    "name": "User100"
                }
            },
            {
                "name": "User4",
                "bestFriend": {
                    "name": "User100"
                }
            },
            {
                "name": "User5",
                "bestFriend": {
                    "name": "U

## There is only one graphene.ObjectType (User) above

## Bug from below, you can see three ObjectTypes: Schools, Classroom and Student.
## In "databse", there is one school, who has two classrooms. Each classroom has two students.

In [7]:
class Student(graphene.ObjectType):
    name = graphene.String()
    
    def __repr__(self):
        return "<Student name={}>".format(self.name)

class Classroom(graphene.ObjectType):
    id = graphene.Int()
    students = graphene.Field(graphene.List(Student))
    
    def __repr__(self):
        return "<Classroom id={}>".format(self.id)
    
    def resolve_students(self, info):
        
        classroom_id_2_student_idss_promise = lambda classroom_id: student_classroom_loader.load_many( [classroom_id] )
        student_ids_2_student_promise = lambda students_ids: student_loader.load_many( students_ids )
        
        
        return classroom_id_2_student_idss_promise(self.id).then(lambda student_idss: student_ids_2_student_promise(student_idss[0]))


class School(graphene.ObjectType):
    id = graphene.Int()
    classrooms = graphene.Field(graphene.List(Classroom))
    
    def resolve_classrooms(self, info):
        classroom_ids = (10, 20)
        print(f'SELECT classroom_id FROM table_classroom_school_relationship WHERE school_id = {self.id}')
        print('    result:', classroom_ids)
        
        # classroom_loader defined below
        return classroom_loader.load_many(classroom_ids)

In [8]:
class SchoolQuery(graphene.ObjectType):
    
    school = graphene.Field(School)

    def resolve_school(self, info):
        the_only_school = School(id = 0)
        return the_only_school

In [9]:
def get_students_from_ids(ids):
    print(f'SELECT * FROM table_student WHERE id IN {ids}')
    result = [Student(name = f'NO.{id}') for id in ids]
    print('    result:', result)
    return result

def get_classrooms_from_ids(ids):
    print(f'SELECT * FROM table_classroom WHERE id IN {ids}')
    result = [Classroom(id = id) for id in ids]
    print('    result:', result)
    return result

def get_student_ids_from_classroom_ids(ids):
    # argument:
    #     classroom_ids with length N
    #     example: [classroom_id_1, classroom_id_2]
    # return:
    #     a list of tuple. Length of list id alse N.
    #     example: [ (student_ids_1, student_ids_2), (student_ids_2, student_ids_3) ]
    #     student_ids_1, student_ids_2 are student_ids for classroom_id_1
    #     student_ids_2, student_ids_3 are student_ids for classroom_id_2
    
    
    print(f'SELECT student_id FROM table_station_classroom_relationship WHERE classroom_id in {ids}')
    
    result = []
    for classroom_id in ids:
        students_ids = (classroom_id + 1, classroom_id + 2)  # each classroom has 2 students
        result.append(students_ids)
        
    print('    result:', result)
    return result

class StudentLoader(DataLoader):
    def batch_load_fn(self, keys):
        # Here we return a promise that will result on the
        # corresponding user for each key in keys
        
        # print(f'StudentLoader.batch_load_fn({keys})')
        return Promise.resolve(get_students_from_ids(ids=keys))

class ClassroomLoader(DataLoader):
    def batch_load_fn(self, keys):
        # Here we return a promise that will result on the
        # corresponding user for each key in keys
        
        # print(f'ClassroomLoader.batch_load_fn({keys})')
        return Promise.resolve(get_classrooms_from_ids(ids=keys))
    

class StudentClassroomLoader(DataLoader):
    def batch_load_fn(self, keys):
        # Here we return a promise that will result on the
        # corresponding user for each key in keys
        
        # print(f'StudentClassroomLoader.batch_load_fn({keys})')
        return Promise.resolve(get_student_ids_from_classroom_ids(ids=keys))
    

student_loader = StudentLoader()
classroom_loader = ClassroomLoader()
student_classroom_loader = StudentClassroomLoader()

In [10]:
run(SchoolQuery, """
{
  school {        # one school
    id
    classrooms {  # N classrooms
      id
      students {  # N*N students
        name
      }
    }
  }
}
""")

SELECT classroom_id FROM table_classroom_school_relationship WHERE school_id = 0
    result: (10, 20)
SELECT * FROM table_classroom WHERE id IN [10, 20]
    result: [<Classroom id=10>, <Classroom id=20>]
SELECT student_id FROM table_station_classroom_relationship WHERE classroom_id in [10, 20]
    result: [(11, 12), (21, 22)]
SELECT * FROM table_student WHERE id IN [11, 12, 21, 22]
    result: [<Student name=NO.11>, <Student name=NO.12>, <Student name=NO.21>, <Student name=NO.22>]
{
    "school": {
        "id": 0,
        "classrooms": [
            {
                "id": 10,
                "students": [
                    {
                        "name": "NO.11"
                    },
                    {
                        "name": "NO.12"
                    }
                ]
            },
            {
                "id": 20,
                "students": [
                    {
                        "name": "NO.21"
                    },
                    {
      